# **Hate Speech Predictor**
## **Import the libraries, load dataset, print shape of data, data description**

In [1]:
!pip install contractions
!pip install autocorrect
import re, string, unicodedata                          # Import Regex, string and unicodedata.
import contractions                                     # Import contractions library.

import numpy as np                                      # Import numpy.
import pandas as pd                                     # Import pandas.
import nltk                                             # Import Natural Language Tool-Kit.

nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.
from sklearn.metrics import accuracy_score

from autocorrect import Speller

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load dataset
train = pd.read_csv('https://github.com/nishantbala/twitter-sentiment-analysis/blob/main/input/train.csv?raw=true')
test = pd.read_csv('https://github.com/nishantbala/twitter-sentiment-analysis/blob/main/input/test.csv?raw=true')

In [3]:
#print shape

train.shape

(31962, 3)

In [4]:
#train data first 5 data
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
#train data description
train.describe()

,id,label
count,31962.000000,31962.000000
mean,15981.500000,0.070146
std,9226.778988,0.255397
min,1.000000,0.000000
25%,7991.250000,0.000000
50%,15981.500000,0.000000
75%,23971.750000,0.000000
max,31962.000000,1.000000


## **Data pre-processing**
### Drop all other columns except “tweet” and “label”

In [6]:
trimmed_data = train.copy(deep=True)
for col in train.columns:
  if (col != 'tweet' and col != 'label'):
    trimmed_data.drop([col], axis=1, inplace=True)
trimmed_data.shape
trimmed_data.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [7]:
# Remove mentions (twitter handle names eg: @user) from the sentence
def remove_mentions(words):
    return " ".join(filter(lambda x:x[0]!='@', words.split()))

trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: remove_mentions(x))
trimmed_data.head()

,label,tweet
0,0,when a father is dysfunctional and is so selfi...
1,0,thanks for #lyft credit i can't use cause they...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ur...
4,0,factsguide: society now #motivation


### Tokenization of tweets

In [8]:
Tokenizer=WhitespaceTokenizer()
trimmed_data['tweet'] = trimmed_data.apply(lambda row: Tokenizer.tokenize(row['tweet']), axis=1)
trimmed_data.head()

,label,tweet
0,0,"[when, a, father, is, dysfunctional, and, is, ..."
1,0,"[thanks, for, #lyft, credit, i, can't, use, ca..."
2,0,"[bihday, your, majesty]"
3,0,"[#model, i, love, u, take, with, u, all, the, ..."
4,0,"[factsguide:, society, now, #motivation]"


### Replace contractions

In [9]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    new_words = []
    for word in text:
      new_word = contractions.fix(word)
      if new_word != '':
        new_words.append(new_word)
    return new_words

trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: replace_contractions(x))

trimmed_data.head()

,label,tweet
0,0,"[when, a, father, is, dysfunctional, and, is, ..."
1,0,"[thanks, for, #lyft, credit, i, cannot, use, b..."
2,0,"[bihday, your, majesty]"
3,0,"[#model, i, love, you, take, with, you, all, t..."
4,0,"[factsguide:, society, now, #motivation]"


### Remove punctuations

In [10]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: remove_punctuation(x))        

trimmed_data.head()

,label,tweet
0,0,"[when, a, father, is, dysfunctional, and, is, ..."
1,0,"[thanks, for, lyft, credit, i, cannot, use, be..."
2,0,"[bihday, your, majesty]"
3,0,"[model, i, love, you, take, with, you, all, th..."
4,0,"[factsguide, society, now, motivation]"


### Convert to lowercase

In [11]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: to_lowercase(x))

trimmed_data.head()

,label,tweet
0,0,"[when, a, father, is, dysfunctional, and, is, ..."
1,0,"[thanks, for, lyft, credit, i, cannot, use, be..."
2,0,"[bihday, your, majesty]"
3,0,"[model, i, love, you, take, with, you, all, th..."
4,0,"[factsguide, society, now, motivation]"


### Stemming

In [12]:
lemmatizer = WordNetLemmatizer()
def lemmatize_list(words):
    new_words = []
    for word in words:
      new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words
trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: lemmatize_list(x))    
trimmed_data.head()

,label,tweet
0,0,"[when, a, father, be, dysfunctional, and, be, ..."
1,0,"[thank, for, lyft, credit, i, cannot, use, bec..."
2,0,"[bihday, your, majesty]"
3,0,"[model, i, love, you, take, with, you, all, th..."
4,0,"[factsguide, society, now, motivation]"


### Autocorrect for misspelt words

In [13]:
spell = Speller(lang='en')
def autospell(words):
  new_words = []
  for word in words:
      new_word = spell(word)
      new_words.append(new_word)
  return new_words
  
#trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: autospell(x))
#trimmed_data.head()

### Normalize words

In [14]:
def normalize(words):   
    return ' '.join(words)
trimmed_data['tweet'] = trimmed_data['tweet'].apply(lambda x: normalize(x))
trimmed_data.head()

,label,tweet
0,0,when a father be dysfunctional and be so selfi...
1,0,thank for lyft credit i cannot use because the...
2,0,bihday your majesty
3,0,model i love you take with you all the time in...
4,0,factsguide society now motivation
